In [1]:
%matplotlib widget

import jax
import jax.numpy as jnp
import jax.tree_util as jtu
import matplotlib.font_manager as font_manager
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from scipy.stats import gaussian_kde
import time
from manim import *

import genjax
from genjax import vi
from genjax import gensp
import numpy as np
import numpy as np
from matplotlib import cm

def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

key = jax.random.PRNGKey(314159)
sns.set_theme(style="white")
label_fontsize = 70  # Set the desired font size here

config.background_color = WHITE
config.disable_caching = True
config.pixel_height = 360
config.pixel_width = 360
config.frame_height = 28.0
config.frame_width = 28.0

In [2]:
@genjax.gen
def model():
    x = vi.normal_reparam(0.0, 10.0) @ "x"
    y = vi.normal_reparam(0.0, 10.0) @ "y"
    v = vi.flip_reinforce(0.7) @ "v"
    rs = jax.lax.cond(v, lambda: (x**2 + y**2), lambda: 0.3 * (x**2 + y**2))
    z = vi.normal_reparam(rs, 0.2 + (rs / 100.0)) @ "z"

data = genjax.choice_map({"z": 5.0})

In [3]:
def importance_samples(key):
    @genjax.gen
    def proposal(tgt):
        x = vi.normal_reparam(0.0, 10.0) @ "x"
        y = vi.normal_reparam(0.0, 10.0) @ "y"
        v = vi.flip_reinforce(0.7) @ "v"
    
    key, sub_key = jax.random.split(key)
    sub_keys = jax.random.split(sub_key, 10000)
    chm_variational = gensp.choice_map_distribution(proposal)
    sir = gensp.CustomImportance(15000, chm_variational)
    scores, v_chm = jax.vmap(sir.random_weighted, in_axes=(0, None))(
        sub_keys, gensp.target(model, (), data)
    )
    chm = v_chm.get_leaf_value()
    
    x, y = chm["x"], chm["y"]
    return (x, y)

def animate_reference_posterior(scene):
    # Create some example (x, y) points
    x_points, y_points = jax.jit(importance_samples)(jax.random.PRNGKey(314159))

    # Set the number of bins
    x_bins = 100
    y_bins = 100

    # Perform kernel density estimate
    xy = np.vstack([x_points, y_points])
    kde = gaussian_kde(xy)
    
    # Define the grid points for the density plot
    x_grid = np.linspace(-10, 10, 100)
    y_grid = np.linspace(-10, 10, 100)
    X, Y = np.meshgrid(x_grid, y_grid)
    positions = np.vstack([X.ravel(), Y.ravel()])
    Z = np.reshape(kde(positions).T, X.shape)

    # Normalize the density values for better visualization
    Z = Z / np.max(Z)

    # Get the viridis colormap from matplotlib
    cmap = plt.get_cmap('viridis')

    # Create a grid of squares to represent the density plot
    squares = VGroup()
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            square = Square(side_length=0.20)
            square.move_to([X[i, j], Y[i, j], 0])
            color = cmap(Z[i, j])
            square.set_fill(rgb_to_color(color), opacity=Z[i, j])
            square.set_stroke(WHITE, width=0.001)
            squares.add(square)

    # Add the grid of squares to the scene
    scene.add(squares)
    
    ring = Circle(radius=jnp.sqrt(5.0), color=RED)
    scene.play(Create(ring))
    text = Text("z = 5.0", color=RED).next_to(ring, 2 * UP).shift(RIGHT)
    scene.play(Write(text))
    #text = Text("High quality approximate posterior", color=BLACK).next_to(ring, 13 * UP)
    #scene.play(Write(text))
    return ring

In [4]:
class ReferencePosteriorWithFlip(Scene):
    def construct(self):
        animate_reference_posterior(self)

%manim -qh ReferencePosteriorWithFlip

Manim Community v0.18.1

/Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/mobject/mobject.py:1175: UserWarning: Explicitly requested dtype float requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  mob.points = mob.points.astype("float")


[06/20/24 16:23:13] INFO     Caching disabled.                                                 ]8;id=586664;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=169881;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

[06/20/24 16:23:14] INFO     Animation 0 : Partial movie file written in                   ]8;id=816245;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=352570;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/partial_movie_files/                         
                             ReferencePosteriorWithFlip/uncached_00000.mp4'                                        

                    INFO     Caching disabled.                                                 ]8;id=867659;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=209826;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

[06/20/24 16:23:15] INFO     Animation 1 : Partial movie file written in                   ]8;id=492051;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=698036;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/partial_movie_files/                         
                             ReferencePosteriorWithFlip/uncached_00001.mp4'                                        

                    INFO     Combining to Movie file.                                      ]8;id=677950;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=579365;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

                    INFO                                                                   ]8;id=817971;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=664224;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#737\737]8;;\
                             File ready at                                                                         
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/ReferencePosteriorWi                         
                             thFlip.mp4'                                                                           
                                                                                                                   

                    INFO     Rendered ReferencePosteriorWithFlip                                       ]8;id=78879;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=987657;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene.py#247\247]8;;\
                             Played 2 animations                                                                   

In [5]:
@genjax.gen
def variational_family(data, ϕ):
    p, μ1, μ2, log_σ1, log_σ2 = ϕ
    p = jax.nn.sigmoid(p)
    v = vi.flip_reinforce(p) @ "v"
    x = vi.normal_reparam(μ1, jnp.exp(log_σ1)) @ "x"
    y = vi.normal_reparam(μ2, jnp.exp(log_σ2)) @ "y"


objective = vi.elbo(model, variational_family, data)

class NaiveGuideTrainingWithFlip(Scene):
    def construct(self):

        # Training.
        key = jax.random.PRNGKey(314159)
        ϕ = (0.5, 0.0, 0.0, 0.0, 0.0)
        jitted = jax.jit(jax.vmap(objective.value_and_grad_estimate, in_axes=(0, None)))
        colormap = cm.get_cmap('viridis')

        ring = animate_reference_posterior(self)

        def block_train(key, ϕ):
            sub_keys = jax.random.split(key, 64)
            (
                loss,
                (
                    _,
                    (
                        _,
                        ϕ_grads,
                    ),
                ),
            ) = jax.vmap(objective.value_and_grad_estimate, in_axes=(0, None))(sub_keys, ((), (data, ϕ)))
            ϕ = jtu.tree_map(lambda v, g: v + 1e-3 * jnp.mean(g), ϕ, ϕ_grads)
            return loss, ϕ

        
        # Warmup
        jitted = jax.jit(block_train)
        key, sub_key = jax.random.split(key)
        _ = jitted(sub_key, ϕ)
        
        total_time = 0.0
        
        for i in range(0, 1501):
            key, sub_key = jax.random.split(key)
            start = time.time()
            loss, ϕ = jitted(sub_key, ϕ)
            duration = time.time() - start
            if i > 2:
                total_time += duration
            
            # Sampling
            if i > 2 and (i % 100 == 0 or i == 3):
                loss_text = Text(f"(evidence lower bound) loss estimate = {trunc(jnp.mean(loss), 2)}", color=BLACK).next_to(ring, 13 * DOWN)
                self.add(loss_text)
                duration_text = Text(f"(training) wall clock time = {trunc(jnp.mean(total_time), 3)} (s)", color=BLACK).next_to(loss_text, DOWN)
                self.add(duration_text)
                key, sub_key = jax.random.split(key)
                sub_keys = jax.random.split(sub_key, 5000)
                tr = jax.jit(jax.vmap(variational_family.simulate, in_axes=(0, None)))(
                    sub_keys, (data, ϕ)
                )
                chm = tr.strip()
                x_data, y_data = chm["x"], chm["y"]
                scores = jnp.exp(tr.get_score())
                colors = [list(colormap(s)) for s in scores]
                new_dots = VGroup(*[Dot(point=(x, y, 1), radius=0.03, color=BLACK) for x, y, c in zip(x_data, y_data, colors)])
                self.add(new_dots)
                self.wait(0.7)
                self.remove(new_dots)
                self.remove(loss_text)
                self.remove(duration_text)
                
%manim -qh NaiveGuideTrainingWithFlip

Manim Community v0.18.1

/Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/mobject/mobject.py:1175: UserWarning: Explicitly requested dtype float requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  mob.points = mob.points.astype("float")


[06/20/24 16:24:28] INFO     Caching disabled.                                                 ]8;id=694516;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=632289;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

[06/20/24 16:24:29] INFO     Animation 0 : Partial movie file written in                   ]8;id=848865;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=113243;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/partial_movie_files/                         
                             NaiveGuideTrainingWithFlip/uncached_00000.mp4'                                        

                    INFO     Caching disabled.                                                 ]8;id=913670;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=397598;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

...

[06/20/24 16:25:13] INFO     Caching disabled.                                                 ]8;id=269270;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=98167;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

[06/20/24 16:25:14] INFO     Animation 17 : Partial movie file written in                  ]8;id=778591;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=686471;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/partial_movie_files/                         
                             NaiveGuideTrainingWithFlip/uncached_00017.mp4'                                        

                    INFO     Combining to Movie file.                                      ]8;id=166302;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=361317;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

                    INFO                                                                   ]8;id=25892;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=750368;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#737\737]8;;\
                             File ready at                                                                         
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/NaiveGuideTrainingWi                         
                             thFlip.mp4'                                                                           
                                                                                                                   

                    INFO     Rendered NaiveGuideTrainingWithFlip                                       ]8;id=415165;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=210276;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene.py#247\247]8;;\
                             Played 18 animations                                                                  

In [6]:
@genjax.gen
def variational_family(data, ϕ):
    p, μ1, μ2, log_σ1, log_σ2 = ϕ
    p = jax.nn.sigmoid(p)
    v = vi.flip_reinforce(p) @ "v"
    x = vi.normal_reparam(μ1, jnp.exp(log_σ1)) @ "x"
    y = vi.normal_reparam(μ2, jnp.exp(log_σ2)) @ "y"


objective = vi.iwae_elbo(model, variational_family, data, 20)

def importance_sampler_with_guide(key, ϕ):
    @genjax.gen
    def hacky_model(ϕ):
        x = vi.normal_reparam(0.0, 10.0) @ "x"
        y = vi.normal_reparam(0.0, 10.0) @ "y"
        v = vi.flip_reinforce(0.8) @ "v"
        rs = jax.lax.cond(v, lambda: (x**2 + y**2), lambda: 0.3 * (x**2 + y**2))
        z = vi.normal_reparam(rs, 0.2 + (rs / 100.0)) @ "z"
    
    
    @genjax.gen
    def hacky_guide(tgt):
        (ϕ,) = tgt.args
        p, μ1, μ2, log_σ1, log_σ2 = ϕ
        p = jax.nn.sigmoid(p)
        v = vi.flip_enum(p) @ "v"
        x = vi.normal_reparam(μ1, jnp.exp(log_σ1)) @ "x"
        y = vi.normal_reparam(μ2, jnp.exp(log_σ2)) @ "y"
    
    sub_keys = jax.random.split(key, 5000)
    chm_variational = gensp.choice_map_distribution(hacky_guide)
    sir = gensp.CustomImportance(20, chm_variational)
    scores, v_chm = jax.vmap(sir.random_weighted, in_axes=(0, None))(
        sub_keys, gensp.target(hacky_model, (ϕ,), data)
    )
    chm = v_chm.get_leaf_value()
    x, y = chm["x"], chm["y"]
    return x, y, scores

class IWELBONaiveGuideTrainingWithFlip(Scene):
    def construct(self):

        # Training.
        key = jax.random.PRNGKey(314159)
        ϕ = (0.5, 0.0, 0.0, 0.0, 0.0)
        jitted = jax.jit(jax.vmap(objective.value_and_grad_estimate, in_axes=(0, None)))
        colormap = cm.get_cmap('viridis')

        ring = animate_reference_posterior(self)

        def block_train(key, ϕ):
            sub_keys = jax.random.split(key, 64)
            (
                loss,
                (
                    _,
                    (
                        _,
                        ϕ_grads,
                    ),
                ),
            ) = jax.vmap(objective.value_and_grad_estimate, in_axes=(0, None))(sub_keys, ((), (data, ϕ)))
            ϕ = jtu.tree_map(lambda v, g: v + 1e-3 * jnp.mean(g), ϕ, ϕ_grads)
            return loss, ϕ

        
        # Warmup
        jitted = jax.jit(block_train)
        key, sub_key = jax.random.split(key)
        _ = jitted(sub_key, ϕ)
        
        total_time = 0.0
        
        for i in range(0, 5001):
            key, sub_key = jax.random.split(key)
            start = time.time()
            loss, ϕ = jitted(sub_key, ϕ)
            duration = time.time() - start
            if i > 2:
                total_time += duration
            
            # Sampling
            if i > 2 and (i % 250 == 0 or i == 3):
                loss_text = Text(f"(evidence lower bound) loss estimate = {trunc(jnp.mean(loss), 2)}", color=BLACK).next_to(ring, 13 * DOWN)
                self.add(loss_text)
                duration_text = Text(f"(training) wall clock time = {trunc(jnp.mean(total_time), 3)} (s)", color=BLACK).next_to(loss_text, DOWN)
                self.add(duration_text)
                
                key, sub_key = jax.random.split(key)
                x_data, y_data, scores = jax.jit(importance_sampler_with_guide)(key, ϕ)
                scores = jnp.exp(scores)
                colors = [list(colormap(s)) for s in scores]
                new_dots = VGroup(*[Dot(point=(x, y, 1), radius=0.03, color=BLACK) for x, y, c in zip(x_data, y_data, colors)])
                self.add(new_dots)
                self.wait(0.7)
                self.remove(new_dots)
                self.remove(loss_text)
                self.remove(duration_text)
                
%manim -qh IWELBONaiveGuideTrainingWithFlip

Manim Community v0.18.1

/Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/mobject/mobject.py:1175: UserWarning: Explicitly requested dtype float requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  mob.points = mob.points.astype("float")


[06/20/24 16:26:00] INFO     Caching disabled.                                                 ]8;id=486987;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=540725;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

[06/20/24 16:26:01] INFO     Animation 0 : Partial movie file written in                   ]8;id=806264;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=675493;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/partial_movie_files/                         
                             IWELBONaiveGuideTrainingWithFlip/uncached_00000.mp4'                                  

                    INFO     Caching disabled.                                                 ]8;id=992120;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=969747;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

...

[06/20/24 16:29:07] INFO     Caching disabled.                                                 ]8;id=696987;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=501966;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

[06/20/24 16:29:08] INFO     Animation 22 : Partial movie file written in                  ]8;id=949805;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=437978;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/partial_movie_files/                         
                             IWELBONaiveGuideTrainingWithFlip/uncached_00022.mp4'                                  

[06/20/24 16:29:09] INFO     Combining to Movie file.                                      ]8;id=464486;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=353495;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

                    INFO                                                                   ]8;id=296438;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=842990;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#737\737]8;;\
                             File ready at                                                                         
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/IWELBONaiveGuideTrai                         
                             ningWithFlip.mp4'                                                                     
                                                                                                                   

                    INFO     Rendered IWELBONaiveGuideTrainingWithFlip                                 ]8;id=134998;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=172368;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene.py#247\247]8;;\
                             Played 23 animations                                                                  

In [7]:
@genjax.gen
def expressive_variational_family(data, ϕ):
    u = vi.uniform() @ "u"
    θ = 2 * jnp.pi * u
    (r1, r2, p, log_σ1, log_σ2, _log_σ1, _log_σ2) = ϕ
    p = jax.nn.sigmoid(p)
    v = vi.flip_reinforce(p) @ "v"
    r = jax.lax.select(v, r1, r2)
    log_σ1 = jax.lax.select(v, log_σ1, _log_σ1)
    log_σ2 = jax.lax.select(v, log_σ2, _log_σ2)
    x = vi.normal_reparam(r * jnp.cos(θ), jnp.exp(log_σ1)) @ "x"
    y = vi.normal_reparam(r * jnp.sin(θ), jnp.exp(log_σ2)) @ "y"


marginal_q = vi.marginal(
    genjax.select("x", "y", "v"), expressive_variational_family, lambda: vi.sir(5)
)

data = genjax.choice_map({"z": 5.0})
hvi_objective = vi.elbo(model, marginal_q, data)

class ExpressiveGuideTrainingWithFlip(Scene):
    def construct(self):
        ring = animate_reference_posterior(self)
        colormap = cm.get_cmap('viridis')

        # Training.
        key = jax.random.PRNGKey(314159)
        ϕ = (1.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        total_time = 0.0

        def block_train(key, ϕ):
            key, sub_key = jax.random.split(key)
            sub_keys = jax.random.split(sub_key, 64)
            loss, (_, ((_, ϕ_grads), ())) = jax.vmap(hvi_objective.value_and_grad_estimate, in_axes=(0, None))(sub_keys, ((), ((data, ϕ), ())))
            ϕ = jtu.tree_map(lambda v, g: v + 1e-3 * jnp.mean(g), ϕ, ϕ_grads)
            return loss, ϕ
        
        # Warmup
        jitted = jax.jit(block_train)
        key, sub_key = jax.random.split(key)
        _ = jitted(sub_key, ϕ)
        
        for i in range(0, 1001):
            key, sub_key = jax.random.split(key)
            start = time.time()
            loss, ϕ = jitted(sub_key, ϕ)
            duration = time.time() - start
            if i > 2:
                total_time += duration
            
            # Sampling
            if i > 0 and (i % 100 == 0 or i == 2):
                loss_text = Text(f"(evidence lower bound) loss estimate = {trunc(jnp.mean(loss), 2)}", color=BLACK).next_to(ring, 13 * DOWN)
                self.add(loss_text)
                duration_text = Text(f"(training) wall clock time = {trunc(jnp.mean(total_time), 3)} (s)", color=BLACK).next_to(loss_text, DOWN)
                self.add(duration_text)
                key, sub_key = jax.random.split(key)
                sub_keys = jax.random.split(sub_key, 5000)
                score_estimates, v_chm = jax.jit(jax.vmap(marginal_q.random_weighted, in_axes=(0, None, None)))(
                    sub_keys, (data, ϕ), ()
                )
                
                chm = v_chm.get_leaf_value()
                x_data, y_data = chm["x"], chm["y"]
                score_estimates = jnp.exp(score_estimates)
                colors = [list(colormap(s)) for s in score_estimates]
                new_dots = VGroup(*[Dot(point=(x, y, 1), radius=0.03, color=BLACK) for x, y, c in zip(x_data, y_data, colors)])
                self.add(new_dots)
                #self.play(Create(new_dots))
                self.wait(0.7)
                self.remove(new_dots)  # Remove previous dots from the scene
                self.remove(loss_text)
                self.remove(duration_text)

%manim -qh ExpressiveGuideTrainingWithFlip

Manim Community v0.18.1

/Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/mobject/mobject.py:1175: UserWarning: Explicitly requested dtype float requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  mob.points = mob.points.astype("float")


[06/20/24 16:30:24] INFO     Caching disabled.                                                 ]8;id=952601;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=207204;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

[06/20/24 16:30:25] INFO     Animation 0 : Partial movie file written in                   ]8;id=56359;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=702796;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/partial_movie_files/                         
                             ExpressiveGuideTrainingWithFlip/uncached_00000.mp4'                                   

                    INFO     Caching disabled.                                                 ]8;id=73554;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=891833;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

...

[06/20/24 16:31:12] INFO     Caching disabled.                                                 ]8;id=283986;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=269868;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

[06/20/24 16:31:14] INFO     Animation 12 : Partial movie file written in                  ]8;id=466249;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=90746;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/partial_movie_files/                         
                             ExpressiveGuideTrainingWithFlip/uncached_00012.mp4'                                   

                    INFO     Combining to Movie file.                                      ]8;id=703520;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=480379;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

                    INFO                                                                   ]8;id=656132;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=410780;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#737\737]8;;\
                             File ready at                                                                         
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/ExpressiveGuideTrain                         
                             ingWithFlip.mp4'                                                                      
                                                                                                                   

                    INFO     Rendered ExpressiveGuideTrainingWithFlip                                  ]8;id=890975;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=577044;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene.py#247\247]8;;\
                             Played 13 animations                                                                  

In [8]:
@genjax.gen
def expressive_variational_family(data, ϕ):
    z = data["z"]
    u = vi.uniform() @ "u"
    θ = 2 * jnp.pi * u
    r, log_σ1, log_σ2 = ϕ
    x = vi.normal_reparam(r * jnp.cos(θ), jnp.exp(log_σ1)) @ "x"
    y = vi.normal_reparam(r * jnp.sin(θ), jnp.exp(log_σ2)) @ "y"

class ExpressiveSamplerWithFlip(Scene):
    def construct(self):
        ring = Circle(radius=jnp.sqrt(5.0), color=RED)
        self.play(Create(ring))
        text = Text("z = 5.0", color=BLACK).next_to(ring, UP).shift(RIGHT)
        self.play(Write(text))
        key = jax.random.PRNGKey(314159)
        ϕ = (4.0, -0.5, -0.5)
        ring2 = Circle(radius=ϕ[0], color=RED)
        for i in range(10):
            key, sub_key = jax.random.split(key)
            tr = expressive_variational_family.simulate(sub_key, (data, ϕ))
            chm = tr.get_choices()
            u, x_data, y_data = chm["u"], chm["x"], chm["y"]
            v = u * 2 * jnp.pi
            arrow = Arrow(start=ORIGIN, end=ring2.point_at_angle(v), color=BLACK)
            self.play(Create(arrow))
            #text = Text("v", color=BLACK).next_to(arrow.get_center(), UP)
            #self.play(Write(text))
            dot = Dot(point=(x_data, y_data, 1), color=BLACK)
            self.play(Create(dot))
            self.wait(0.3)
            self.remove(dot)
            self.remove(arrow)

In [9]:
%manim -qh ExpressiveSamplerWithFlip

Manim Community v0.18.1

/Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/mobject/mobject.py:1175: UserWarning: Explicitly requested dtype float requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  mob.points = mob.points.astype("float")


                    INFO     Caching disabled.                                                 ]8;id=21496;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=514973;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

                    INFO     Animation 0 : Partial movie file written in                   ]8;id=271218;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=189643;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/partial_movie_files/                         
                             ExpressiveSamplerWithFlip/uncached_00000.mp4'                                         

                    INFO     Caching disabled.                                                 ]8;id=124148;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=631243;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

...

                    INFO     Caching disabled.                                                 ]8;id=291104;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=482307;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/renderer/cairo_renderer.py#78\78]8;;\

                    INFO     Animation 31 : Partial movie file written in                  ]8;id=705306;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=145625;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/partial_movie_files/                         
                             ExpressiveSamplerWithFlip/uncached_00031.mp4'                                         

                    INFO     Combining to Movie file.                                      ]8;id=867768;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=939577;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

[06/20/24 16:31:31] INFO                                                                   ]8;id=290237;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=80117;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#737\737]8;;\
                             File ready at                                                                         
                             '/Users/mccoybecker/research/programmable-vi-pldi-2024/media/                         
                             videos/programmable-vi-pldi-2024/1080p60/ExpressiveSamplerWit                         
                             hFlip.mp4'                                                                            
                                                                                                                   

                    INFO     Rendered ExpressiveSamplerWithFlip                                        ]8;id=795833;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=373344;file:///Users/mccoybecker/miniconda3/envs/pldi2024/lib/python3.10/site-packages/manim/scene/scene.py#247\247]8;;\
                             Played 32 animations                                                                  